In [30]:
import os
import boto3
import gzip
import shutil
import polars as pl
from botocore.config import Config
from bet_edge.data_io.env_cred_provider import EnvCredProvider

# Initialize a session using your credentials
ecp = EnvCredProvider()
creds = ecp.get_credentials()

session = boto3.Session(
    aws_access_key_id=creds["polygon_access_key_id"],
    aws_secret_access_key=creds["polygon_secret_access_key"],
)

# Create an S3 client
s3 = session.client(
    "s3",
    endpoint_url="https://files.polygon.io",
    config=Config(signature_version="s3v4"),
)

# Set bucket name and prefix
bucket_name = "flatfiles"
prefix = "us_stocks_sip/day_aggs_v1"  # Only process 'day_aggs_v1' data

# Define local storage path
base_local_dir = os.path.join("..", "..", "data_lake")

# Initialize paginator for listing S3 objects
paginator = s3.get_paginator("list_objects_v2")

# Process files
for page in paginator.paginate(Bucket=bucket_name, Prefix=prefix):
    for obj in page.get("Contents", []):
        object_key = obj["Key"]

        # Skip files that are not .csv.gz
        if not object_key.endswith(".csv.gz"):
            continue

        try:
            # Construct full local file path preserving folder structure
            local_file_path = os.path.join(base_local_dir, *object_key.split("/"))
            decompressed_file_path = local_file_path.replace(".gz", "")
            parquet_file_path = decompressed_file_path.replace(".csv", ".parquet")

            # Ensure directory structure exists
            os.makedirs(os.path.dirname(local_file_path), exist_ok=True)

            # Download file from S3
            print(f"Downloading: {object_key}")
            s3.download_file(bucket_name, object_key, local_file_path)

            # Decompress the file
            print(f"Decompressing: {local_file_path}")
            with gzip.open(local_file_path, "rb") as f_in:
                with open(decompressed_file_path, "wb") as f_out:
                    shutil.copyfileobj(f_in, f_out)

            # Convert CSV to Parquet
            print(f"Converting to Parquet: {parquet_file_path}")
            df = pl.read_csv(decompressed_file_path)
            df.write_parquet(parquet_file_path)

            # Clean up - remove original compressed and decompressed CSV
            os.remove(local_file_path)
            os.remove(decompressed_file_path)

            print(f"Stored as Parquet: {parquet_file_path}\n")

        except Exception as e:
            print(f"Error processing {object_key}: {e}")

Downloading: us_stocks_sip/day_aggs_v1/2003/09/2003-09-10.csv.gz
Error processing us_stocks_sip/day_aggs_v1/2003/09/2003-09-10.csv.gz: An error occurred (403) when calling the HeadObject operation: Forbidden
Downloading: us_stocks_sip/day_aggs_v1/2003/09/2003-09-11.csv.gz
Error processing us_stocks_sip/day_aggs_v1/2003/09/2003-09-11.csv.gz: An error occurred (403) when calling the HeadObject operation: Forbidden
Downloading: us_stocks_sip/day_aggs_v1/2003/09/2003-09-12.csv.gz
Error processing us_stocks_sip/day_aggs_v1/2003/09/2003-09-12.csv.gz: An error occurred (403) when calling the HeadObject operation: Forbidden
Downloading: us_stocks_sip/day_aggs_v1/2003/09/2003-09-15.csv.gz
Error processing us_stocks_sip/day_aggs_v1/2003/09/2003-09-15.csv.gz: An error occurred (403) when calling the HeadObject operation: Forbidden
Downloading: us_stocks_sip/day_aggs_v1/2003/09/2003-09-16.csv.gz
Error processing us_stocks_sip/day_aggs_v1/2003/09/2003-09-16.csv.gz: An error occurred (403) when call

In [3]:
%load_ext autoreload
%autoreload 2

In [31]:
import os


def remove_empty_dirs(root_dir):
    """
    Recursively removes all empty subdirectories from the given root directory.
    """
    for dirpath, dirnames, filenames in os.walk(root_dir, topdown=False):
        for dirname in dirnames:
            dir_full_path = os.path.join(dirpath, dirname)
            if not os.listdir(dir_full_path):  # Check if directory is empty
                os.rmdir(dir_full_path)
                print(f"Deleted empty folder: {dir_full_path}")


# Define your base directory
data_lake_path = os.path.join("..", "..", "data_lake")

# Remove empty directories
remove_empty_dirs(data_lake_path)

Deleted empty folder: ..\..\data_lake\us_stocks_sip\day_aggs_v1\2003\09
Deleted empty folder: ..\..\data_lake\us_stocks_sip\day_aggs_v1\2003\10
Deleted empty folder: ..\..\data_lake\us_stocks_sip\day_aggs_v1\2003\11
Deleted empty folder: ..\..\data_lake\us_stocks_sip\day_aggs_v1\2003\12
Deleted empty folder: ..\..\data_lake\us_stocks_sip\day_aggs_v1\2004\01
Deleted empty folder: ..\..\data_lake\us_stocks_sip\day_aggs_v1\2004\02
Deleted empty folder: ..\..\data_lake\us_stocks_sip\day_aggs_v1\2004\03
Deleted empty folder: ..\..\data_lake\us_stocks_sip\day_aggs_v1\2004\04
Deleted empty folder: ..\..\data_lake\us_stocks_sip\day_aggs_v1\2004\05
Deleted empty folder: ..\..\data_lake\us_stocks_sip\day_aggs_v1\2004\06
Deleted empty folder: ..\..\data_lake\us_stocks_sip\day_aggs_v1\2004\07
Deleted empty folder: ..\..\data_lake\us_stocks_sip\day_aggs_v1\2004\08
Deleted empty folder: ..\..\data_lake\us_stocks_sip\day_aggs_v1\2004\09
Deleted empty folder: ..\..\data_lake\us_stocks_sip\day_aggs_v1\

In [60]:
import os
import polars as pl

# Define base directory
data_lake_path = os.path.join("..", "..", "data_lake")

# Example: Load a specific Parquet file (update this to match your dataset)
file_path = os.path.join(data_lake_path, "us_stocks_sip", "day_aggs_v1", "2023", "04", "2023-04-10.parquet")

# Check if file exists
if os.path.exists(file_path):
    # Load the file with Polars
    sdf = pl.read_parquet(file_path)

    # Show basic info
    print(df.head())  # Preview first few rows
    print(df.schema)  # Show column names and types
else:
    print(f"File not found: {file_path}")

shape: (5, 8)
┌────────┬─────────┬────────┬────────┬─────────┬────────┬─────────────────────┬──────────────┐
│ ticker ┆ volume  ┆ open   ┆ close  ┆ high    ┆ low    ┆ window_start        ┆ transactions │
│ ---    ┆ ---     ┆ ---    ┆ ---    ┆ ---     ┆ ---    ┆ ---                 ┆ ---          │
│ str    ┆ i64     ┆ f64    ┆ f64    ┆ f64     ┆ f64    ┆ i64                 ┆ i64          │
╞════════╪═════════╪════════╪════════╪═════════╪════════╪═════════════════════╪══════════════╡
│ A      ┆ 1102780 ┆ 137.38 ┆ 138.02 ┆ 138.08  ┆ 135.86 ┆ 1681099200000000000 ┆ 18607        │
│ AA     ┆ 2438434 ┆ 39.16  ┆ 39.78  ┆ 40.15   ┆ 39.0   ┆ 1681099200000000000 ┆ 24267        │
│ AAA    ┆ 20      ┆ 24.47  ┆ 24.47  ┆ 24.47   ┆ 24.47  ┆ 1681099200000000000 ┆ 1            │
│ AAAU   ┆ 1993119 ┆ 19.76  ┆ 19.75  ┆ 19.7773 ┆ 19.656 ┆ 1681099200000000000 ┆ 1960         │
│ AAC    ┆ 272951  ┆ 10.32  ┆ 10.345 ┆ 10.36   ┆ 10.32  ┆ 1681099200000000000 ┆ 463          │
└────────┴─────────┴────────┴───────

In [63]:
data_lake_path = os.path.join("..", "..", "data_lake")
print(os.path.abspath(data_lake_path))

c:\Users\Ashle\OneDrive\Documents\data_lake


In [61]:
sdf

ticker,volume,open,close,high,low,window_start,transactions
str,i64,f64,f64,f64,f64,i64,i64
"""A""",1102780,137.38,138.02,138.08,135.86,1681099200000000000,18607
"""AA""",2438434,39.16,39.78,40.15,39.0,1681099200000000000,24267
"""AAA""",20,24.47,24.47,24.47,24.47,1681099200000000000,1
"""AAAU""",1993119,19.76,19.75,19.7773,19.656,1681099200000000000,1960
"""AAC""",272951,10.32,10.345,10.36,10.32,1681099200000000000,463
…,…,…,…,…,…,…,…
"""ZWZZT""",205177,41.0,41.0,41.0,41.0,1681099200000000000,1
"""ZXIET""",1000,100.0,100.0,100.0,100.0,1681099200000000000,1
"""ZYME""",595611,9.44,9.37,9.49,9.28,1681099200000000000,5829


In [75]:
df = df.with_columns(pl.col("window_start").cast(pl.Datetime("ns")))
df

ticker,volume,open,close,high,low,window_start,transactions
str,i64,f64,f64,f64,f64,datetime[ns],i64
"""A""",1102780,137.38,138.02,138.08,135.86,2023-04-10 04:00:00,18607
"""AA""",2438434,39.16,39.78,40.15,39.0,2023-04-10 04:00:00,24267
"""AAA""",20,24.47,24.47,24.47,24.47,2023-04-10 04:00:00,1
"""AAAU""",1993119,19.76,19.75,19.7773,19.656,2023-04-10 04:00:00,1960
"""AAC""",272951,10.32,10.345,10.36,10.32,2023-04-10 04:00:00,463
…,…,…,…,…,…,…,…
"""ZWZZT""",205177,41.0,41.0,41.0,41.0,2023-04-10 04:00:00,1
"""ZXIET""",1000,100.0,100.0,100.0,100.0,2023-04-10 04:00:00,1
"""ZYME""",595611,9.44,9.37,9.49,9.28,2023-04-10 04:00:00,5829


In [52]:
# Updated regex pattern without lookahead.
pattern = r"^O:(?P<underlying>.*?)(?P<exp_date>\d{6})" r"(?P<opt_type>[CP])(?P<strike>\d{8})$"

# Extract ticker parts into a struct column.
tdf = df.with_columns(pl.col("ticker").str.extract(pattern).alias("parsed"))

# Expand the struct into separate columns.
# tdf = df.with_columns([
#    pl.col("parsed").struct.field("underlying").alias("underlying"),
#    pl.col("parsed").struct.field("exp_date").alias("exp_date"),
#    pl.col("parsed").struct.field("opt_type").alias("opt_type"),
#    pl.col("parsed").struct.field("strike").alias("strike_str")
# ]).drop("parsed")
print(tdf)
# Convert expiration date from YYMMDD to a proper date.
"""
df = df.with_columns(
    (
        "20" + pl.col("exp_date").str.slice(0, 2) + "-" +
        pl.col("exp_date").str.slice(2, 2) + "-" +
        pl.col("exp_date").str.slice(4, 2)
    ).str.strptime(pl.Date, format="%Y-%m-%d").alias("expiration_date")
)


# Convert strike price: cast to integer and divide by 1000.
df = df.with_columns(
    (pl.col("strike_str").cast(pl.Int64) / 1000).alias("strike_price")
)

# Optionally, drop the intermediate columns.
df = df.drop(["exp_date", "strike_str"]) """

shape: (176_852, 9)
┌───────────────────┬────────┬──────┬───────┬───┬──────┬───────────────────┬──────────────┬────────┐
│ ticker            ┆ volume ┆ open ┆ close ┆ … ┆ low  ┆ window_start      ┆ transactions ┆ parsed │
│ ---               ┆ ---    ┆ ---  ┆ ---   ┆   ┆ ---  ┆ ---               ┆ ---          ┆ ---    │
│ str               ┆ i64    ┆ f64  ┆ f64   ┆   ┆ f64  ┆ i64               ┆ i64          ┆ str    │
╞═══════════════════╪════════╪══════╪═══════╪═══╪══════╪═══════════════════╪══════════════╪════════╡
│ O:A230421C0014000 ┆ 3      ┆ 1.15 ┆ 1.41  ┆ … ┆ 1.15 ┆ 16810992000000000 ┆ 3            ┆ A      │
│ 0                 ┆        ┆      ┆       ┆   ┆      ┆ 00                ┆              ┆        │
│ O:A230421C0014500 ┆ 1      ┆ 0.25 ┆ 0.25  ┆ … ┆ 0.25 ┆ 16810992000000000 ┆ 1            ┆ A      │
│ 0                 ┆        ┆      ┆       ┆   ┆      ┆ 00                ┆              ┆        │
│ O:A230421C0015500 ┆ 347    ┆ 0.05 ┆ 0.05  ┆ … ┆ 0.04 ┆ 16810992000000

'\ndf = df.with_columns(\n    (\n        "20" + pl.col("exp_date").str.slice(0, 2) + "-" +\n        pl.col("exp_date").str.slice(2, 2) + "-" +\n        pl.col("exp_date").str.slice(4, 2)\n    ).str.strptime(pl.Date, format="%Y-%m-%d").alias("expiration_date")\n)\n\n\n# Convert strike price: cast to integer and divide by 1000.\ndf = df.with_columns(\n    (pl.col("strike_str").cast(pl.Int64) / 1000).alias("strike_price")\n)\n\n# Optionally, drop the intermediate columns.\ndf = df.drop(["exp_date", "strike_str"]) '

In [55]:
def process_option_tickers(df: pl.DataFrame) -> pl.DataFrame:
    """
    Processes a DataFrame containing OPRA-style option tickers.

    Expects a 'ticker' column in the input DataFrame with tickers in the format:
      O:<underlying><exp_date><opt_type><strike>
    where:
      - underlying: variable length (letters and digits) after "O:" and before the last 15 characters
      - exp_date: 6 digits (YYMMDD)
      - opt_type: 'C' or 'P'
      - strike: 8 digits (strike price multiplied by 1000)

    Returns a new DataFrame with the following additional columns:
      - underlying: the underlying asset extracted from the ticker
      - expiration_date: the expiration date as a Date type (assumed to be 20YY-MM-DD)
      - option_type: 'Call' or 'Put'
      - strike_price: the strike price as a float (strike / 1000)
    """
    # Calculate the length of each ticker
    df = df.with_columns(pl.col("ticker").str.len_chars().alias("ticker_length"))

    # The last 15 characters always represent exp_date (6), option type (1) and strike (8)
    # Underlying is what remains after the "O:" prefix (2 characters) and before the last 15 characters.
    df = df.with_columns(
        # Underlying: start at index 2; length = ticker_length - 2 - 15
        pl.col("ticker").str.slice(2, pl.col("ticker_length") - 2 - 15).alias("underlying"),
        # Expiration date: 6 characters starting at position ticker_length - 15
        pl.col("ticker").str.slice(pl.col("ticker_length") - 15, 6).alias("exp_date"),
        # Option type: 1 character starting at position ticker_length - 9
        pl.col("ticker").str.slice(pl.col("ticker_length") - 9, 1).alias("opt_type"),
        # Strike: the last 8 characters
        pl.col("ticker").str.slice(-8, 8).alias("strike_str"),
    )

    # Remove the temporary ticker_length column
    df = df.drop("ticker_length")

    # Convert the expiration date from YYMMDD to a proper date.
    # Assumes years are in 2000's.
    df = df.with_columns(
        (
            "20"
            + pl.col("exp_date").str.slice(0, 2)
            + "-"
            + pl.col("exp_date").str.slice(2, 2)
            + "-"
            + pl.col("exp_date").str.slice(4, 2)
        )
        .str.strptime(pl.Date, format="%Y-%m-%d")
        .alias("expiration_date")
    )

    # Convert strike price to a float (dividing by 1000)
    df = df.with_columns((pl.col("strike_str").cast(pl.Int64) / 1000).alias("strike_price"))

    # Optionally drop intermediate columns
    df = df.drop(["exp_date", "strike_str"])

    return df


tdf = process_option_tickers(df)
tdf

ticker,volume,open,close,high,low,window_start,transactions,underlying,opt_type,expiration_date,strike_price
str,i64,f64,f64,f64,f64,i64,i64,str,str,date,f64
"""O:A230421C00140000""",3,1.15,1.41,1.41,1.15,1681099200000000000,3,"""A""","""C""",2023-04-21,140.0
"""O:A230421C00145000""",1,0.25,0.25,0.25,0.25,1681099200000000000,1,"""A""","""C""",2023-04-21,145.0
"""O:A230421C00155000""",347,0.05,0.05,0.05,0.04,1681099200000000000,37,"""A""","""C""",2023-04-21,155.0
"""O:A230421P00125000""",2,0.3,0.28,0.3,0.28,1681099200000000000,2,"""A""","""P""",2023-04-21,125.0
"""O:A230421P00130000""",10,0.75,0.51,0.8,0.51,1681099200000000000,6,"""A""","""P""",2023-04-21,130.0
…,…,…,…,…,…,…,…,…,…,…,…
"""O:ZYME231020P00010000""",5,2.0,2.0,2.0,2.0,1681099200000000000,1,"""ZYME""","""P""",2023-10-20,10.0
"""O:ZYXI230421P00010000""",5,0.1,0.1,0.1,0.1,1681099200000000000,1,"""ZYXI""","""P""",2023-04-21,10.0
"""O:ZYXI230519C00012500""",3,0.95,0.6,0.95,0.6,1681099200000000000,2,"""ZYXI""","""C""",2023-05-19,12.5


In [59]:
tdf.select(pl.col("underlying")).unique()

underlying
str
"""BHVN"""
"""EQT"""
"""ASPN"""
"""LIZI"""
"""MTW"""
…
"""AROW"""
"""VTI"""
"""GFI"""


In [29]:
import boto3
from botocore.config import Config
from bet_edge.data_io.env_cred_provider import EnvCredProvider

# Initialize credentials
ecp = EnvCredProvider()
creds = ecp.get_credentials()

# Initialize a boto3 session
session = boto3.Session(
    aws_access_key_id=creds["polygon_access_key_id"],
    aws_secret_access_key=creds["polygon_secret_access_key"],
)

# Create an S3 client
s3 = session.client(
    "s3",
    endpoint_url="https://files.polygon.io",
    config=Config(signature_version="s3v4"),
)

# Specify prefix
prefix = "us_options_opra"

# Initialize paginator
paginator = s3.get_paginator("list_objects_v2")

# Extract second path segment from keys
second_segments = set()
for page in paginator.paginate(Bucket="flatfiles", Prefix=prefix):
    for obj in page.get("Contents", []):
        key_parts = obj["Key"].split("/")
        if len(key_parts) > 1:  # Ensure there is a second path segment
            second_segments.add(key_parts[1])

# Print distinct values
print(second_segments)

{'minute_aggs_v1', 'quotes_v1', 'trades_v1', 'day_aggs_v1'}


In [65]:
from bet_edge.data_io.data_lake.load_parquet import load_data_for_day

date_str = "2023-04-10"  # Set the date for which you want to load data.
stock_df, options_df = load_data_for_day(date_str)

In [ ]:
from bet_edge.options_pipeline.options_processing import process_option_tickers

options_df = process_option_tickers(options_df)

ticker,volume,open,close,high,low,window_start,transactions,underlying,opt_type,expiration_date,strike_price
str,i64,f64,f64,f64,f64,i64,i64,str,str,date,f64
"""O:A230421C00140000""",3,1.15,1.41,1.41,1.15,1681099200000000000,3,"""A""","""C""",2023-04-21,140.0
"""O:A230421C00145000""",1,0.25,0.25,0.25,0.25,1681099200000000000,1,"""A""","""C""",2023-04-21,145.0
"""O:A230421C00155000""",347,0.05,0.05,0.05,0.04,1681099200000000000,37,"""A""","""C""",2023-04-21,155.0
"""O:A230421P00125000""",2,0.3,0.28,0.3,0.28,1681099200000000000,2,"""A""","""P""",2023-04-21,125.0
"""O:A230421P00130000""",10,0.75,0.51,0.8,0.51,1681099200000000000,6,"""A""","""P""",2023-04-21,130.0
…,…,…,…,…,…,…,…,…,…,…,…
"""O:ZYME231020P00010000""",5,2.0,2.0,2.0,2.0,1681099200000000000,1,"""ZYME""","""P""",2023-10-20,10.0
"""O:ZYXI230421P00010000""",5,0.1,0.1,0.1,0.1,1681099200000000000,1,"""ZYXI""","""P""",2023-04-21,10.0
"""O:ZYXI230519C00012500""",3,0.95,0.6,0.95,0.6,1681099200000000000,2,"""ZYXI""","""C""",2023-05-19,12.5


In [72]:
tdf = options_df.join(
    stock_df, how="left", left_on=["underlying", "window_start"], right_on=["ticker", "window_start"]
).rename(
    {
        "volume_right": "stock_volume",
        "open_right": "stock_open",
        "close_right": "stock_close",
        "high_right": "stock_high",
        "low_right": "stock_low",
        "transactions_right": "stock_transactions",
    }
)
tdf.columns

['ticker',
 'volume',
 'open',
 'close',
 'high',
 'low',
 'window_start',
 'transactions',
 'underlying',
 'opt_type',
 'expiration_date',
 'strike_price',
 'stock_volume',
 'stock_open',
 'stock_close',
 'stock_high',
 'stock_low',
 'stock_transactions']

In [81]:
import yfinance as yf

ticker_obj = yf.Ticker("T")
dividends = ticker_obj.dividends

print(type(dividends))

<class 'pandas.core.series.Series'>


In [74]:
import yfinance as yf


def get_dividend_yield(symbol: str) -> float:
    """
    Retrieves the dividend yield for a given underlying symbol using yfinance.
    Returns 0 if not available.
    """
    try:
        ticker = yf.Ticker(symbol)
        info = ticker.info
        # dividendYield can be None; if so, default to 0.
        return info.get("dividendYield") or 0.0
    except Exception as e:
        # In production, you might log this error.
        return 0.0


get_dividend_yield("T")

4.09

In [8]:
import requests
import pandas as pd
import os
from datetime import datetime

from bet_edge.data_io.env_cred_provider import EnvCredProvider

# Initialize credentials
ecp = EnvCredProvider()
creds = ecp.get_credentials()


def get_all_us_common_stock_tickers(api_key: str):
    url = "https://api.polygon.io/v3/reference/tickers"
    tickers = []
    params = {"market": "stocks", "type": "CS", "apiKey": api_key, "limit": 1000}

    while url:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        tickers.extend([item["ticker"] for item in data.get("results", [])])
        url = data.get("next_url")
        params = {"apiKey": api_key} if url else None

    return tickers


def get_dividends_for_ticker(ticker: str, api_key: str):
    url = "https://api.polygon.io/v3/reference/dividends"
    params = {"ticker": ticker, "apiKey": api_key}

    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()

    dividends = data.get("results", [])
    return dividends


def save_dividends_to_lake(dividends: list, base_path: str):
    if not dividends:
        return

    df = pd.DataFrame(dividends)
    df["pay_date"] = pd.to_datetime(df["pay_date"])

    for year, yearly_df in df.groupby(df["pay_date"].dt.year):
        for ticker, ticker_df in yearly_df.groupby("ticker"):
            folder = f"{base_path}/{year}"
            os.makedirs(folder, exist_ok=True)
            file_path = f"{base_path}/{year}/{ticker}.parquet"
            yearly_ticker_df = ticker_df.reset_index(drop=True)
            yearly_ticker_df.to_parquet(file_path, index=False)
            print(f"Saved {ticker} dividends for {year}")


base_dir = r"c:\Users\Ashle\OneDrive\Documents\data_lake\dividends"

tickers = get_all_us_common_stock_tickers(creds["polygon_api_key"])

for ticker in tickers:
    try:
        dividends = get_dividends_for_ticker(ticker, creds["polygon_api_key"])
        save_dividends_to_lake(dividends, base_path=base_dir)
    except requests.HTTPError as e:
        print(f"Failed to fetch dividends for {ticker}: {e}")

Saved A dividends for 2023
Saved A dividends for 2024
Saved A dividends for 2025
Saved AA dividends for 2022
Saved AA dividends for 2023
Saved AA dividends for 2024
Saved AA dividends for 2025
Saved AAL dividends for 2017
Saved AAL dividends for 2018
Saved AAL dividends for 2019
Saved AAL dividends for 2020
Saved AAME dividends for 2014
Saved AAME dividends for 2015
Saved AAME dividends for 2016
Saved AAME dividends for 2017
Saved AAME dividends for 2018
Saved AAME dividends for 2019
Saved AAME dividends for 2021
Saved AAME dividends for 2022
Saved AAME dividends for 2023
Saved AAME dividends for 2024
Saved AAMI dividends for 2025
Saved AAON dividends for 2022
Saved AAON dividends for 2023
Saved AAON dividends for 2024
Saved AAON dividends for 2025
Saved AAP dividends for 2023
Saved AAP dividends for 2024
Saved AAP dividends for 2025
Saved AAPL dividends for 2022
Saved AAPL dividends for 2023
Saved AAPL dividends for 2024
Saved AAPL dividends for 2025
Saved AAT dividends for 2022
Saved

In [10]:
from bet_edge.options_pipeline.data_lake import load_dividend_data

div = load_dividend_data("ADBE", 2004)

div

cash_amount,currency,dividend_type,ex_dividend_date,frequency,id,pay_date,record_date,ticker
f64,str,str,datetime[μs],i64,str,datetime[ns],str,str
0.0125,"""USD""","""CD""",2004-09-24 00:00:00,0,"""E6e8906cf74fb4901f46b58d4ef512…",2004-10-12 00:00:00,"""2004-09-28""","""ADBE"""
0.0125,"""USD""","""CD""",2004-06-25 00:00:00,0,"""E7854bca4a7a43d63d5d77447ad15a…",2004-07-13 00:00:00,"""2004-06-29""","""ADBE"""
0.0125,"""USD""","""CD""",2004-03-26 00:00:00,0,"""E1f60870c7c5ec819d56f012ea203e…",2004-04-13 00:00:00,"""2004-03-30""","""ADBE"""
0.0125,"""USD""","""CD""",2004-01-07 00:00:00,0,"""Ec83f068760f9988b776864b1d9b99…",2004-01-23 00:00:00,"""2004-01-09""","""ADBE"""


In [14]:
import os
import polars as pl

# Base directory where dividend files are stored in a partitioned (yearly) structure.
BASE_DIR = r"c:\Users\Ashle\OneDrive\Documents\data_lake\dividends"  # e.g., dividends/year=2023/AAPL.parquet
# Output directory to store merged dividend history by ticker.
OUTPUT_DIR = r"c:\Users\Ashle\OneDrive\Documents\data_lake\dividends_by_ticker"

# Create output directory if it does not exist.
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Dictionary to hold file paths for each ticker.
ticker_files = {}

# Walk through the partitioned directories
for subdir in os.listdir(BASE_DIR):
    # We expect subdirectories to be named like "year=2023", "year=2022", etc.
    year_dir = os.path.join(BASE_DIR, subdir)
    if os.path.isdir(year_dir):
        # List all Parquet files in the year directory.
        for filename in os.listdir(year_dir):
            if filename.endswith(".parquet"):
                # Assume file names are like "AAPL.parquet"; extract ticker symbol.
                ticker = os.path.splitext(filename)[0]
                file_path = os.path.join(year_dir, filename)
                ticker_files.setdefault(ticker, []).append(file_path)

# Now, for each ticker, merge its files and store a single file.
for ticker, file_list in ticker_files.items():
    dfs = []
    for file_path in file_list:
        try:
            df = pl.read_parquet(file_path)
            dfs.append(df)
        except Exception as e:
            print(f"Error reading {file_path} for ticker {ticker}: {e}")
    if dfs:
        # Concatenate all dividend history data for this ticker.
        merged_df = pl.concat(dfs, how="vertical_relaxed")
        # Optionally sort by the dividend event date if the column exists.
        if "ex_dividend_date" in merged_df.columns:
            merged_df = merged_df.sort("ex_dividend_date")
        # Write the merged dividend history to a single Parquet file.
        output_file = os.path.join(OUTPUT_DIR, f"{ticker}.parquet")
        merged_df.write_parquet(output_file)
        print(f"Merged {len(file_list)} files for ticker {ticker} into {output_file}")
    else:
        print(f"No data found for ticker {ticker}.")

Merged 3 files for ticker ADBE into c:\Users\Ashle\OneDrive\Documents\data_lake\dividends_by_ticker\ADBE.parquet
Merged 7 files for ticker AZ into c:\Users\Ashle\OneDrive\Documents\data_lake\dividends_by_ticker\AZ.parquet
Merged 4 files for ticker BOX into c:\Users\Ashle\OneDrive\Documents\data_lake\dividends_by_ticker\BOX.parquet
Merged 2 files for ticker BVS into c:\Users\Ashle\OneDrive\Documents\data_lake\dividends_by_ticker\BVS.parquet
Merged 3 files for ticker GP into c:\Users\Ashle\OneDrive\Documents\data_lake\dividends_by_ticker\GP.parquet
Merged 1 files for ticker HAYW into c:\Users\Ashle\OneDrive\Documents\data_lake\dividends_by_ticker\HAYW.parquet
Merged 2 files for ticker LITE into c:\Users\Ashle\OneDrive\Documents\data_lake\dividends_by_ticker\LITE.parquet
Merged 4 files for ticker TEM into c:\Users\Ashle\OneDrive\Documents\data_lake\dividends_by_ticker\TEM.parquet
Merged 2 files for ticker ADSK into c:\Users\Ashle\OneDrive\Documents\data_lake\dividends_by_ticker\ADSK.parqu